### Code to test the model 

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, out_dim, dp_rate):
        super(MLP, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, input_dim*2)
        self.output_layer = nn.Linear(input_dim*2, out_dim)
        self.dropout = nn.Dropout(dp_rate)
        self.relu = torch.nn.ReLU()
       
    def forward(self, x_in):
        z1 = self.dropout(x_in) # output of the input layer, after dropout
        z2 = self.relu(self.hidden_layer(z1)) # output of the hidden layer
        logits = self.output_layer(z2)
        return logits

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

path_of_downloaded_files = "/users/richa/glove.6B/glove.6B.300d.txt"
glove_file = datapath(path_of_downloaded_files)
word2vec_glove_file = get_tmpfile("glove.6B.300d.txt")
glove2word2vec(glove_file, word2vec_glove_file)
word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
from nltk.tokenize import word_tokenize
def vectorize_sent(word_vectors, sent, oov_vec):
    word_vecs = []
    for token in word_tokenize(sent): 
        if token not in word_vectors: 
            word_vecs.append(oov_vec)
        else:
            word_vecs.append(word_vectors[token].astype('float64'))
    return np.mean(word_vecs,axis=0)

In [ ]:
# reconstruct trained model from pickle

import pickle
def reconstruct_model(pickle_path):
    saved_model_dic = pickle.load(open(pickle_path,"rb"))
    input_dim = saved_model_dic['input_dim']
    dp_rate = saved_model_dic['dropout_rate']
    output_dim = 2
    model = MLP(input_dim, output_dim, dp_rate)
    saved_weights = saved_model_dic['neural_weights']
    n_epochs = saved_model_dic['n_epochs']
    batch_size = saved_model_dic['batch_size']
    lr = saved_model_dic['lr']
    model.load_state_dict(saved_weights, batch_size)
    oov_vec = saved_model_dic['oov_vector']
    
    return model, oov_vec

In [ ]:
# use the reconstructed model to make predictions on the test data

def test_trained_model(model, oov_vec, test_text):
    test_vecs = [vectorize_sent(word_vectors, ss, oov_vec ) for ss in test_text]
    test_vecs_tensor = torch.tensor(test_vecs, dtype=torch.float)
    test_prediction = model(test_vecs_tensor)
    pred_labels = [np.argmax(tp.detach().numpy()) for tp in test_prediction]
    return pred_labels

In [ ]:
# load sample test data

import pandas as pd
import numpy as np
test_data = pd.read_table('coursework2_train.tsv')
test_text = test_data['sentence_text'].tolist()[-2000:]
test_raw_labels = test_data['label'].tolist()[-2000:]
label_dic = {'non-propaganda':0, 'propaganda':1} 
test_labels = [label_dic[rl] for rl in test_raw_labels]

print('test data size:', len(test_labels))

# reconstruct model and make predictions

model, oov_vec = reconstruct_model('trained_model.pickle')
test_pred = test_trained_model(model, oov_vec, test_text)

# test model

from sklearn.metrics import precision_recall_fscore_support,accuracy_score
pre, rec, f1, _ = precision_recall_fscore_support(test_labels, test_pred, average='macro')
print('macro-F1 on test data:', f1)